In [ ]:
## YOLOv5

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/siim/version2/
!ls

/content/drive/MyDrive/siim/version2


In [ ]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 9194, done.
remote: Total 9194 (delta 0), reused 0 (delta 0), pack-reused 9194
Receiving objects: 100% (9194/9194), 9.66 MiB | 9.01 MiB/s, done.
Resolving deltas: 100% (6379/6379), done.
/content/drive/My Drive/siim/version2/yolov5
     |████████████████████████████████| 3.0 MB 8.7 MB/s 
     |████████████████████████████████| 636 kB 60.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla P100-PCIE-16GB)


In [ ]:
# Access WANDB account
!pip install -q --upgrade wandb
 
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 7.6 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 170 kB 56.5 MB/s 
     |████████████████████████████████| 133 kB 31.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/siim/tmp/train/"
IMG_SIZE = 256
BATCH_SIZE = 16
EPOCHS = 10

In [ ]:
import tensorflow as tf

# Get image path from image_id
def get_path(image_id):
    path = tf.io.gfile.glob(TRAIN_PATH + f"*{image_id}.jpg")[0]
    return path

image_dict = {
    "opacity" : 1,
    "none" : 0
}
df = pd.read_csv("/content/drive/MyDrive/siim/summary/train_info.csv")

df["image_label"] = df["image_label"].map(lambda x : x.split(" ")[0])
df["image_label_id"] = df["image_label"].map(lambda x  : image_dict[x])
df["filepath"] = df["image_id"].map(get_path)
df.head()

,image_id,study_id,pneumonia,pneumonia_class,height,width,boxes,image_label,image_label_id,filepath
0,000a312787f2,5776db0cec75,Typical Appearance,1,3488,4256,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity,1,/content/drive/MyDrive/siim/tmp/train/000a3127...
1,000c3a3f293f,ff0879eb20ed,Negative for Pneumonia,0,2320,2832,"[{'x': 0, 'y': 0, 'width': 1, 'height': 1}]",none,0,/content/drive/MyDrive/siim/tmp/train/000c3a3f...
2,0012ff7358bc,9d514ce429a7,Typical Appearance,1,2544,3056,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity,1,/content/drive/MyDrive/siim/tmp/train/0012ff73...
3,001398f4ff4f,28dddc8559b2,Atypical Appearance,3,3520,4280,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity,1,/content/drive/MyDrive/siim/tmp/train/001398f4...
4,001bd15d1891,dfd9fdd85a3e,Typical Appearance,1,2800,3408,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity,1,/content/drive/MyDrive/siim/tmp/train/001bd15d...


In [ ]:
train_df,val_df = train_test_split(df,
                                    test_size=0.2,
                                    random_state = 42,
                                    stratify = df.image_label.values
                                    )

train_df.loc[:,"data"] = "train"
val_df.loc[:,"data"] = "val"
df = pd.concat([train_df, val_df]).reset_index(drop=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


NameError: ignored

In [ ]:
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(val_df)}')

Size of dataset: 6334, training images: 5067. validation images: 1267


In [ ]:
!ls

CONTRIBUTING.md  Dockerfile  LICENSE	requirements.txt  utils
data		 export.py   models	train.py	  val.py
detect.py	 hubconf.py  README.md	tutorial.ipynb


In [ ]:
os.makedirs('/content/drive/MyDrive/siim/version2/dataset/images/train', exist_ok=True)
os.makedirs('/content/drive/MyDrive/siim/version2/dataset/images/val', exist_ok=True)

os.makedirs('/content/drive/MyDrive/siim/version2/dataset/labels/train', exist_ok=True)
os.makedirs('/content/drive/MyDrive/siim/version2/dataset/labels/val', exist_ok=True)
print("Created folder structure")

Created folder structure


In [ ]:
print(df.iloc[1,9],df.iloc[1,])

/content/drive/MyDrive/siim/tmp/train/8b2544f4361c.jpg image_id                                                8b2544f4361c
study_id                                                7d04273b5886
pneumonia                                         Typical Appearance
pneumonia_class                                                    1
height                                                          3480
width                                                           4248
boxes              [{'x': 597.81637, 'y': 1014.4836, 'width': 515...
image_label                                                  opacity
image_label_id                                                     1
filepath           /content/drive/MyDrive/siim/tmp/train/8b2544f4...
data                                                           train
Name: 1, dtype: object


In [ ]:
# Move the images to relevant split folder.
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    if row.data == 'train':
        copyfile(row.filepath, f'/content/drive/MyDrive/siim/version2/dataset/images/train/{row.image_id}.jpg')
    else:
        copyfile(row.filepath, f'/content/drive/MyDrive/siim/version2/dataset/images/val/{row.image_id}.jpg')

100%|██████████| 6334/6334 [29:41<00:00,  3.55it/s]


In [ ]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '/content/drive/MyDrive/siim/version2/dataset/images/train',
    val = '/content/drive/MyDrive/siim/version2/dataset/images/val',
    nc = 2,
    names = ['none', 'opacity']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('/content/drive/MyDrive/siim/version2/yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat /content/drive/MyDrive/siim/version2/yolov5/data/data.yaml

{names: [none, opacity], nc: 2, train: /content/drive/MyDrive/siim/version2/dataset/images/train,
  val: /content/drive/MyDrive/siim/version2/dataset/images/val}


In [ ]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.width
    scale_y = IMG_SIZE/row.height
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

In [ ]:
train_image_level=pd.read_csv('/content/drive/MyDrive/siim/summary/train_image_level.csv')
# Modify values in the id column
train_image_level['image_id'] = train_image_level.apply(lambda row: row.id.split('_')[0], axis=1)
# Get image level labels
train_image_level['image_level'] = train_image_level.apply(lambda row: row.label.split(' ')[0], axis=1)
train_image_level.head()

,id,boxes,label,StudyInstanceUID,image_id,image_level
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,000a312787f2,opacity
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,000c3a3f293f,none
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0012ff7358bc,opacity
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,001398f4ff4f,opacity
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,001bd15d1891,opacity


In [ ]:
df = df.merge(train_image_level, on='image_id',how="left")
df.head(2)

,image_id,study_id,pneumonia,pneumonia_class,height,width,boxes_x,image_label,image_label_id,filepath,data,id,boxes_y,label,StudyInstanceUID,image_level
0,b9175a64ad09,587699c00f43,Indeterminate Appearance,2,2618,3005,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity,1,/content/drive/MyDrive/siim/tmp/train/b9175a64...,train,b9175a64ad09_image,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity 1 2244.66714 1575.60367 2504.06533 178...,587699c00f43,opacity
1,8b2544f4361c,7d04273b5886,Typical Appearance,1,3480,4248,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity,1,/content/drive/MyDrive/siim/tmp/train/8b2544f4...,train,8b2544f4361c_image,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity 1 597.81637 1014.4836 1113.13638 2448....,7d04273b5886,opacity


In [ ]:
# Prepare the txt files for bounding box
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.image_id
    # Get split
    split = row.data
    # Get image-level label
    label = row.image_level
    
    if row.data=='train':
        file_name = f'/content/drive/MyDrive/siim/version2/dataset/labels/train/{row.image_id}.txt'
    else:
        file_name = f'/content/drive/MyDrive/siim/version2/dataset/labels/val/{row.image_id}.txt'
        
    
    if label=='opacity':
        # Get bboxes
        bboxes = get_bbox(row)
        # Scale bounding boxes
        scale_bboxes = scale_bbox(row, bboxes)
        # Format for YOLOv5
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
        with open(file_name, 'w') as f:
            for bbox in yolo_bboxes:
                bbox = [1]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')

100%|██████████| 6334/6334 [00:26<00:00, 241.00it/s]


In [ ]:
%cd /content/drive/MyDrive/siim/version2/yolov5/


/content/drive/MyDrive/siim/version2/yolov5


In [ ]:
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project version2

100% 755k/755k [00:00<00:00, 16.3MB/s]
train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=version2, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 2021-9-5 torch 1.9.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degre

In [ ]:
TEST_PATH = "/content/drive/MyDrive/siim/tmp/test"
BEST_MODEL_PATH  ="/content/drive/MyDrive/siim/version2/yolov5/version2/exp/weights/best.pt"

In [ ]:
!python detect.py --weights {BEST_MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img 256 \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf

detect: weights=['/content/drive/MyDrive/siim/version2/yolov5/version2/exp/weights/best.pt'], source=/content/drive/MyDrive/siim/tmp/test, imgsz=[256, 256], conf_thres=0.281, iou_thres=0.5, max_det=3, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-407-g5487451 torch 1.9.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients, 16.3 GFLOPs
image 1/1263 /content/drive/MyDrive/siim/tmp/test/0026720152f5.jpg: 256x256 Done. (0.010s)
image 2/1263 /content/drive/MyDrive/siim/tmp/test/00fc8fc35dc1.jpg: 256x256 2 opacitys, Done. (0.010s)
image 3/1263 /content/drive/MyDrive/siim/tmp/test/01c3512eebc3.jpg: 256x256 Done. (0.008s)
image 4/1263 /content/drive/MyDrive/siim/tmp/test/01f948

In [ ]:
PRED_PATH = "runs/detect/exp6/labels/"
PRED_FILES = os.listdir(PRED_PATH)

print("Sample prediction (in txt file) : \n")

with open(PRED_PATH + PRED_FILES[0], "r") as f:
    ls = f.read().strip("\n").split(" ")
    print(f"LABEL : {ls[0]} \nX_CENTER : {ls[1]} \nY_CENTER : {ls[2]} \nWIDTH : {ls[3]} \nHEIGHT : {ls[4]} \nCONFIDENCE : {ls[5]}")

Sample prediction (in txt file) : 

LABEL : 1 
X_CENTER : 0.783203 
Y_CENTER : 0.373047 
WIDTH : 0.300781 
HEIGHT : 0.410156 
CONFIDENCE : 0.510527
1
